In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, f1_score, accuracy_score
import numpy as np
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

In [2]:
# Load EfficientNetB0 without pretrained weights
base_model = EfficientNetB0(weights=None, include_top=False, input_shape=(224, 224, 3))

In [3]:
# Build the model
model = models.Sequential([
    base_model,
    #layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    #layers.MaxPooling2D((2, 2)),
    #layers.Dropout(0.2),
    #layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    #layers.MaxPooling2D((2, 2)),
    #layers.Dropout(0.2),
    #layers.Flatten(),
    #layers.Dense(256, activation='relu'),
    #layers.Dropout(0.2),
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')  # For binary classification
])

In [4]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,213,668 (16.07 MB)

 Trainable params: 4,171,645 (15.91 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [6]:
# Paths to your data
train_dir = "Training_data"
val_dir = "Validation_data"

In [7]:
# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [8]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 5012 images belonging to 2 classes.


In [10]:
val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 420 images belonging to 2 classes.


In [11]:
# Learning rate scheduler
lr_schedule = LearningRateScheduler(
    lambda epoch: 1e-3 * 10**(-epoch / 20), verbose=1
)

In [12]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True,
    verbose=1
)

In [13]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model.keras', 
    monitor='val_accuracy', 
    save_best_only=True, 
    mode='max'
)

In [14]:
# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=50,
    callbacks=[lr_schedule, early_stopping, checkpoint]
)

C:\Users\PMLS\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/50


KeyboardInterrupt: 

In [14]:
# Load the best model
best_model = tf.keras.models.load_model('best_model.keras')

In [15]:
# Evaluate on validation data
val_preds = best_model.predict(val_data)
val_labels = val_data.classes
val_preds = (val_preds > 0.4).astype(int).flatten()

C:\Users\PMLS\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


14/14 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step


In [16]:
# Calculate metrics
accuracy = accuracy_score(val_labels, val_preds)
f1 = f1_score(val_labels, val_preds)
report = classification_report(val_labels, val_preds, target_names=['Negative', 'Positive'])

print(f"Validation Accuracy: {accuracy}")
print(f"Validation F1 Score: {f1}")
print("Classification Report:\n", report)

Validation Accuracy: 0.5119047619047619
Validation F1 Score: 0.42896935933147634
Classification Report:
               precision    recall  f1-score   support

    Negative       0.51      0.66      0.57       210
    Positive       0.52      0.37      0.43       210

    accuracy                           0.51       420
   macro avg       0.51      0.51      0.50       420
weighted avg       0.51      0.51      0.50       420



In [18]:
from flask import Flask, request, render_template, redirect, url_for
import numpy as np
import cv2
from tensorflow.keras.models import load_model
#from crop_brain_contour import crop_brain_contour  # Assuming you already have this function

app = Flask(__name__)

# Load the pre-trained model
model = load_model('best_model.keras')

def prepare_image(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert to RGB if the model requires RGB inputs
    if model.input_shape[-1] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Resize and normalize
    image = cv2.resize(image, (model.input_shape[1], model.input_shape[2]))
    image = image / 255.0  # Normalize pixel values to [0, 1]
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

@app.route('/')
def index():
    return render_template('./index.html')



@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return redirect(request.url)

    file = request.files['image']
    if file.filename == '':
        return redirect(request.url)

    # Save the uploaded file temporarily in the static directory
    image_path = f'static/uploaded_image.jpg'
    file.save(image_path)

    # Prepare the image and make the prediction
    image = prepare_image(image_path)
    prediction = model.predict(image)

    # Interpret the result
    result = "Tumor Positive" if prediction > 0.5 else "Tumor Negative"
    prediction_value = float(prediction[0][0])  # Extract the prediction value

    # Pass the result, prediction value, and image path to the template
    return render_template(
        'result.html', 
        result=result, 
        prediction_value=prediction_value, 
        image_path='uploaded_image.jpg'
    )


if __name__ == '__main__':
    app.run(debug=False)  # Set debug=False


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Jan/2025 18:36:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2025 18:36:47] "GET /static/styles.css HTTP/1.1" 304 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


127.0.0.1 - - [09/Jan/2025 18:36:59] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2025 18:36:59] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Jan/2025 18:36:59] "GET /static/uploaded_image.jpg HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step


127.0.0.1 - - [09/Jan/2025 18:37:12] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2025 18:37:12] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Jan/2025 18:37:12] "GET /static/uploaded_image.jpg HTTP/1.1" 200 -
